## Parte 1 ● Ejecutar los scripts con una muestra relativamente pequeña de cada uno de losendpoints y verificar que todo funcione correctamente.

○ Demanda 

○ Balance

https://www.ree.es/es/datos/apidatos




In [3]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

numpy==2.1.3
pandas==2.2.3
requests==2.32.3


In [6]:
import requests
import pandas as pd

# Constantes comunes
URL = "https://apidatos.ree.es/es/datos/balance/balance-electrico"
HEADERS = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es",
    "User-Agent": "Mozilla/5.0"
}

# Fechas y truncamiento
START_DATE = "2023-01-01T00:00"
END_DATE = "2023-12-31T23:59"
TIME_TRUNC = "day"

# Diccionario de geo_ids y nombres de regiones
region_ids = {
    "8741": "peninsular",
    "8742": "canarias",
    "8743": "baleares",
    "8744": "ceuta",
    "8745": "melilla",
    "4": "andalucia",
    "5": "aragon",
    "6": "cantabria",
    "7": "castilla la mancha",
    "8": "castilla y leon",
    "9": "cataluña",
    "10": "pais vasco",
    "11": "principado de asturias",
    "13": "comunidad de madrid",
    "14": "comunidad de navarra",
    "15": "comunidad de valenciana",
    "16": "extremadura",
    "17": "galicia",
    "20": "la rioja",
    "21": "region de murcia"
}

# DataFrame final
df_total = pd.DataFrame()

# Loop para hacer las consultas
for geo_id, region_name in region_ids.items():
    params = {
        "start_date": START_DATE,
        "end_date": END_DATE,
        "time_trunc": TIME_TRUNC,
        "geo_id": geo_id
    }

    response = requests.get(URL, headers=HEADERS, params=params)
    data = response.json()
    included = data.get('included', [])

    registros = []

    for grupo in included:
        group_name = grupo['attributes']['title']
        contenidos = grupo['attributes'].get('content', [])

        for item in contenidos:
            indicador = item['attributes']['title']
            valores = item['attributes'].get('values', [])

            for punto in valores:
                registros.append({
                    'fecha': punto['datetime'],
                    'valor': punto['value'],
                    'tipo': indicador,
                    'energia': group_name,
                    'region': region_name
                })

    df = pd.DataFrame(registros)
    df_total = pd.concat([df_total, df], ignore_index=True)

# Guardar en un único archivo CSV
df_total.to_csv("balance_electrico_dia_2023_todas_regiones.csv", index=False)
print("Datos guardados en balance_electrico_dia_2023_todas_regiones.csv")


Datos guardados en balance_electrico_dia_2023_todas_regiones.csv


In [263]:
################################################################################################################################